In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import RidgeClassifier

In [2]:
df = pd.read_csv('roberta_eng_abs_ext.csv')

In [3]:
X = df.iloc[:,:-1]

In [4]:
X

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.045318,0.106291,-0.003215,-0.067849,0.027385,-0.127473,-0.029526,0.032465,0.032632,-0.086068,...,0.039978,-0.050516,-0.105249,-0.060616,0.014517,0.111777,0.058287,-0.074195,0.009236,0.020355
1,-0.051724,0.116939,0.019695,-0.105494,0.033264,-0.070311,-0.006279,0.061977,0.007398,-0.091103,...,0.012504,-0.015868,-0.088793,-0.080782,0.007715,0.105455,0.058583,-0.040270,0.008042,-0.013936
2,-0.010546,0.108884,0.001479,-0.138257,0.051355,-0.135540,0.008098,0.051294,0.010994,-0.042763,...,-0.014196,0.014642,-0.096850,-0.071336,-0.050518,0.096773,0.024355,-0.091703,-0.011808,0.005190
3,-0.054942,0.087816,0.019472,-0.118021,0.059199,-0.082324,-0.024870,0.060929,0.012410,-0.098716,...,0.020057,-0.011953,-0.098715,-0.092882,-0.004579,0.122452,-0.007312,-0.053264,-0.000205,0.005530
4,-0.026756,0.077807,-0.000185,-0.094653,0.057685,-0.071129,-0.000685,0.097987,0.053599,-0.065829,...,0.005963,-0.013227,-0.050602,-0.054803,-0.005757,0.072578,0.028736,-0.040133,-0.029626,0.037816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,-0.053095,0.064454,-0.004253,-0.104838,0.048685,-0.087474,-0.045931,0.078595,0.043535,-0.087520,...,-0.012299,-0.036790,-0.049352,-0.086227,-0.014976,0.133873,0.006647,-0.087797,-0.025291,-0.017844
1196,-0.063583,0.077264,-0.016701,-0.090798,0.050827,-0.063856,0.008944,0.049084,0.035193,-0.085438,...,0.043240,0.014395,-0.050048,-0.067256,-0.007649,0.127047,0.003745,-0.057350,-0.016512,-0.033022
1197,-0.055323,0.113551,-0.003577,-0.086809,0.031325,-0.086053,-0.001449,0.020454,0.028360,-0.108519,...,0.003348,0.000723,-0.065087,-0.053158,0.001449,0.102973,0.003530,-0.079374,-0.016366,-0.006790
1198,-0.061254,0.088840,-0.007258,-0.079009,0.032370,-0.083520,-0.024850,0.059002,0.040812,-0.100361,...,-0.010307,-0.014336,-0.042034,-0.059780,-0.005447,0.107997,0.042148,-0.090695,-0.007495,-0.009438


In [5]:
Y = df.iloc[:,-1]

In [6]:
Y

0       1
1       2
2       0
3       0
4       3
       ..
1195    3
1196    2
1197    2
1198    3
1199    3
Name: Judgement Status, Length: 1200, dtype: int64

In [7]:
pca = PCA()
X_pca = pca.fit_transform(X)

In [8]:
X_pca

array([[-9.82113371e-02,  2.50828417e-01,  5.60698911e-02, ...,
        -1.28383848e-04, -3.95741317e-05, -3.82615102e-08],
       [ 1.31875106e-01,  1.14299430e-01,  5.57400182e-02, ...,
         2.42525922e-04,  3.45240338e-06, -7.54342530e-08],
       [-1.19125517e-01, -6.37560107e-02, -6.27694673e-04, ...,
         3.13395055e-06,  4.39140800e-05, -1.10092436e-08],
       ...,
       [ 1.21561546e-01, -3.83859561e-02, -2.15583834e-02, ...,
         5.59842238e-04,  2.11851363e-05,  4.52324123e-08],
       [ 1.17762394e-01,  9.83129913e-03, -1.03422947e-02, ...,
        -3.72481125e-04, -3.63934376e-05,  2.60192217e-08],
       [ 2.20326534e-01,  5.02193272e-03, -1.37922984e-01, ...,
         1.26180986e-04, -3.96551669e-05,  2.21856442e-08]])

In [9]:
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
print("Cumulative explained variance:", cumulative_variance)

Cumulative explained variance: [0.20919963 0.2821137  0.33323283 0.37552085 0.40786158 0.4382193
 0.46293037 0.48424002 0.50305142 0.52103937 0.53777955 0.55344138
 0.56776754 0.58000064 0.59164118 0.60252957 0.61249615 0.62149683
 0.62997512 0.63819644 0.64609865 0.65379507 0.66048447 0.66703962
 0.6733655  0.67962926 0.68563725 0.69119642 0.6966105  0.7019257
 0.70719198 0.71228804 0.71721511 0.72207122 0.72678822 0.73125381
 0.73557267 0.73975055 0.74387499 0.7478848  0.7518179  0.75565624
 0.7593314  0.76299536 0.76652371 0.77001591 0.77341531 0.77672471
 0.7800032  0.78321625 0.78632531 0.78932914 0.79232304 0.79518751
 0.79804354 0.80085026 0.80359178 0.80625118 0.80886146 0.81143505
 0.81394848 0.8164154  0.81883498 0.82120336 0.8235361  0.82586
 0.82811836 0.83034418 0.83255159 0.83465476 0.83674007 0.83875138
 0.8407487  0.84272224 0.84464959 0.84655707 0.84844862 0.8503277
 0.85218822 0.85402877 0.85581078 0.85756918 0.85930704 0.86102755
 0.86272356 0.86441327 0.86608608 0.8

In [10]:
n_components = np.argmax(cumulative_variance >= 0.95) + 1
print(f"Number of components to retain 95% variance: {n_components}")

Number of components to retain 95% variance: 175


In [11]:
pca = PCA(n_components=n_components)
X_reduced = pca.fit_transform(X)

In [12]:
X_reduced_df = pd.DataFrame(X_reduced)

In [13]:
X_reduced_df

,0,1,2,3,4,5,6,7,8,9,...,165,166,167,168,169,170,171,172,173,174
0,-0.098211,0.250828,0.056070,0.172745,0.158913,-0.096079,-0.066711,0.021703,0.018244,0.012974,...,0.029902,0.008030,-0.020076,0.001261,-0.001349,0.010657,-0.008160,-0.033211,0.010565,0.008326
1,0.131875,0.114299,0.055740,0.062140,0.050495,0.007590,0.007530,0.038001,-0.146042,0.054359,...,0.002341,0.008152,-0.011090,-0.010254,0.001725,0.012204,0.007563,-0.012619,-0.005881,-0.007012
2,-0.119126,-0.063756,-0.000628,0.032426,0.224285,-0.188183,-0.108031,-0.055691,-0.140759,0.107719,...,-0.018633,0.007112,-0.044911,-0.014514,-0.019671,0.003846,-0.042511,0.008975,0.014113,0.011101
3,0.347256,0.292587,0.223981,0.157487,0.208781,-0.055681,-0.002916,-0.072127,-0.109315,0.048299,...,-0.003437,-0.017252,0.012005,0.000691,-0.017790,0.020095,0.024952,-0.012248,-0.024914,0.010880
4,-0.277805,-0.073339,0.208072,-0.018170,0.064474,0.047829,-0.155506,0.142461,-0.018100,0.094815,...,0.012251,-0.000601,-0.001111,-0.001606,-0.001809,0.001003,-0.035835,0.000321,0.009033,-0.005304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,0.097995,0.026680,-0.048025,-0.082330,0.048514,-0.097055,0.089675,0.137547,-0.122957,-0.096259,...,-0.010395,0.002943,0.041972,-0.016956,-0.002368,0.003058,0.016414,0.009645,0.014411,-0.025138
1196,0.512149,-0.045441,0.275050,-0.058645,-0.132395,-0.029935,-0.023297,-0.080648,-0.006814,-0.010707,...,-0.024441,0.003226,-0.016556,0.018204,0.013710,-0.002111,-0.034179,0.011006,-0.004524,-0.024708
1197,0.121562,-0.038386,-0.021558,-0.069058,-0.010863,-0.207825,-0.083849,-0.057232,-0.070052,-0.009371,...,0.015913,0.005078,0.004195,-0.021110,-0.008158,-0.010268,-0.007479,0.005264,-0.004251,0.006326
1198,0.117762,0.009831,-0.010342,-0.182199,0.096732,-0.046663,0.063776,0.010267,-0.225068,-0.053179,...,-0.016511,-0.018427,0.000159,-0.013170,-0.013049,0.012880,0.001700,0.011874,0.034886,0.030563


In [14]:
X_train,X_test,Y_train,Y_test = train_test_split(X_reduced_df,Y,test_size=0.2,random_state=42)

In [23]:
models = {
    "KNN": KNeighborsClassifier(),
    "SVC": SVC(probability=True, random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(eval_metric='logloss', random_state=42)
}

# Train and evaluate each model
results = {}

for name, model in models.items():
    model.fit(X_train, Y_train)  # Train the model
    y_pred = model.predict(X_test)  # Predict on the test set
    accuracy = accuracy_score(Y_test, y_pred)  # Calculate accuracy
    results[name] = accuracy  # Store the result

# Print the results
print("Model Accuracies:")
for model, accuracy in results.items():
    print(f"{model}: {accuracy:.4f}")

Model Accuracies:
KNN: 0.4667
SVC: 0.5292
Logistic Regression: 0.4708
Decision Tree: 0.3583
Random Forest: 0.4500
XGBoost: 0.4583


In [24]:
base_models = [
    ('svc', SVC(kernel='linear', probability=True, random_state=42)),        # SVM with linear kernel
    ('lr', LogisticRegression(max_iter=5000, random_state=42)),              # Linear model
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),       # Tree-based model
    ('knn', KNeighborsClassifier(n_neighbors=5)),                            # K-Nearest Neighbors
    ('et', ExtraTreesClassifier(n_estimators=100, random_state=42)),         # Extra Trees Classifier
    ('xgb', XGBClassifier(eval_metric='logloss', random_state=42)),          # XGBoost
    ('ada', AdaBoostClassifier(n_estimators=100, random_state=42)),          # AdaBoost
]

In [25]:
meta_model = GradientBoostingClassifier(random_state=42)

In [18]:
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5, n_jobs=-1)

In [20]:
stacking_model.fit(X_train, Y_train)

/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent thi

StackingClassifier(cv=5,
                   estimators=[('svc',
                                SVC(kernel='linear', probability=True,
                                    random_state=42)),
                               ('lr',
                                LogisticRegression(max_iter=5000,
                                                   random_state=42)),
                               ('rf', RandomForestClassifier(random_state=42)),
                               ('knn', KNeighborsClassifier()),
                               ('et', ExtraTreesClassifier(random_state=42)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsamp...
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=42, ...)),
                               ('ada',
                                AdaBoostClassifier(n_estimators=100,
                                                   random_state=42))],
                   final_estimator=GradientBoostingClassifier(random_state=42),
                   n_jobs=-1)

In [21]:
y_pred_175 = stacking_model.predict(X_test)

In [22]:
print((accuracy_score(Y_test,y_pred_175)))

0.475


In [26]:
correlation_matrix = pd.DataFrame(X_reduced_df).corr()  

In [27]:
threshold = 0.9
high_corr_pairs = np.where(np.abs(correlation_matrix) > threshold)
high_corr_pairs = [(correlation_matrix.index[x], correlation_matrix.columns[y]) 
                   for x, y in zip(*high_corr_pairs) if x != y and x < y]
print("Highly correlated pairs:", high_corr_pairs)

Highly correlated pairs: []


In [39]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

# Define the parameter grid for RandomizedSearchCV
param_grid = {
    'lr__C': np.logspace(-4, 4, 10),  # Logistic Regression regularization strength
    'rf__n_estimators': [100, 200, 300, 400],
    'rf__max_depth': [10, 20, 30, None],
    'svc__C': np.logspace(-4, 4, 10),  # SVM regularization strength
    'svc__kernel': ['linear', 'rbf'],
    'knn__n_neighbors': [3, 5, 7, 9],
    'et__n_estimators': [100, 200, 300],
    'xgb__n_estimators': [100, 200, 300],
    'xgb__learning_rate': [0.01, 0.1, 0.3],
    'ada__n_estimators': [50, 100, 200],
    'ada__learning_rate': [0.01, 0.1, 0.5],
    'mlp__hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'mlp__alpha': [0.0001, 0.001, 0.01]
}

# Set up the base models with diverse types
base_models = [
    ('lr', LogisticRegression(max_iter=5000, random_state=42)),
    ('rf', RandomForestClassifier(random_state=42)),
    ('svc', SVC(probability=True, random_state=42)),
    ('knn', KNeighborsClassifier()),
    ('et', ExtraTreesClassifier(n_estimators=100, random_state=42)),
    ('xgb', XGBClassifier(eval_metric='logloss', random_state=42)),
    ('ada', AdaBoostClassifier(random_state=42)),
    ('ridge', RidgeClassifier()),  # Ridge Classifier
    ('mlp', MLPClassifier(max_iter=1000, random_state=42))  # Neural Network
]

# Meta-model: XGBoost as a strong learner
meta_model = XGBClassifier(eval_metric='logloss', random_state=42)

# Set up the stacking model
stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5
)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    stacking_model,
    param_distributions=param_grid,
    n_iter=50,  # Number of iterations for the random search
    cv=5,
    random_state=42,
    n_jobs=-1,
    verbose=2
)

# Load and scale your data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Assuming X_train, X_test, y_train, y_test are already defined
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train and evaluate the model with randomized search
random_search.fit(X_train_scaled, Y_train)

# Best parameters found by RandomizedSearchCV
print("Best parameters found: ", random_search.best_params_)

# Evaluate the best model
train_accuracy = random_search.score(X_train_scaled, Y_train)
test_accuracy = random_search.score(X_test_scaled, Y_test)

print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


Fitting 5 folds for each of 50 candidates, totalling 250 fits


/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent thi

[CV] END ada__learning_rate=0.5, ada__n_estimators=100, et__n_estimators=200, knn__n_neighbors=5, lr__C=2.782559402207126, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(50,), rf__max_depth=None, rf__n_estimators=100, svc__C=0.005994842503189409, svc__kernel=linear, xgb__learning_rate=0.3, xgb__n_estimators=300; total time=  49.0s
[CV] END ada__learning_rate=0.5, ada__n_estimators=100, et__n_estimators=200, knn__n_neighbors=5, lr__C=2.782559402207126, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(50,), rf__max_depth=None, rf__n_estimators=100, svc__C=0.005994842503189409, svc__kernel=linear, xgb__learning_rate=0.3, xgb__n_estimators=300; total time=  49.8s
[CV] END ada__learning_rate=0.5, ada__n_estimators=100, et__n_estimators=200, knn__n_neighbors=5, lr__C=2.782559402207126, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(50,), rf__max_depth=None, rf__n_estimators=100, svc__C=0.005994842503189409, svc__kernel=linear, xgb__learning_rate=0.3, xgb__n_estimators=300; total time=  50.4s
[CV] END a

/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent thi

[CV] END ada__learning_rate=0.1, ada__n_estimators=200, et__n_estimators=100, knn__n_neighbors=7, lr__C=0.0001, mlp__alpha=0.01, mlp__hidden_layer_sizes=(50, 50), rf__max_depth=20, rf__n_estimators=200, svc__C=166.81005372000558, svc__kernel=rbf, xgb__learning_rate=0.1, xgb__n_estimators=100; total time= 1.3min
[CV] END ada__learning_rate=0.1, ada__n_estimators=200, et__n_estimators=100, knn__n_neighbors=7, lr__C=0.0001, mlp__alpha=0.01, mlp__hidden_layer_sizes=(50, 50), rf__max_depth=20, rf__n_estimators=200, svc__C=166.81005372000558, svc__kernel=rbf, xgb__learning_rate=0.1, xgb__n_estimators=100; total time= 1.3min
[CV] END ada__learning_rate=0.1, ada__n_estimators=200, et__n_estimators=100, knn__n_neighbors=7, lr__C=0.0001, mlp__alpha=0.01, mlp__hidden_layer_sizes=(50, 50), rf__max_depth=20, rf__n_estimators=200, svc__C=166.81005372000558, svc__kernel=rbf, xgb__learning_rate=0.1, xgb__n_estimators=100; total time= 1.3min
[CV] END ada__learning_rate=0.1, ada__n_estimators=200, et__n

...

[CV] END ada__learning_rate=0.1, ada__n_estimators=100, et__n_estimators=100, knn__n_neighbors=9, lr__C=0.0001, mlp__alpha=0.01, mlp__hidden_layer_sizes=(50,), rf__max_depth=20, rf__n_estimators=300, svc__C=10000.0, svc__kernel=linear, xgb__learning_rate=0.01, xgb__n_estimators=200; total time=138.9min


/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent thi

[CV] END ada__learning_rate=0.1, ada__n_estimators=100, et__n_estimators=100, knn__n_neighbors=9, lr__C=0.0001, mlp__alpha=0.01, mlp__hidden_layer_sizes=(50,), rf__max_depth=20, rf__n_estimators=300, svc__C=10000.0, svc__kernel=linear, xgb__learning_rate=0.01, xgb__n_estimators=200; total time=105.4min


/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent thi

Best parameters found:  {'xgb__n_estimators': 200, 'xgb__learning_rate': 0.3, 'svc__kernel': 'rbf', 'svc__C': 0.005994842503189409, 'rf__n_estimators': 400, 'rf__max_depth': 20, 'mlp__hidden_layer_sizes': (50,), 'mlp__alpha': 0.01, 'lr__C': 1291.5496650148827, 'knn__n_neighbors': 3, 'et__n_estimators': 200, 'ada__n_estimators': 50, 'ada__learning_rate': 0.01}
Train Accuracy: 0.9990
Test Accuracy: 0.6208


##### Best parameters found:  {'xgb__n_estimators': 200, 'xgb__learning_rate': 0.3, 'svc__kernel': 'rbf', 'svc__C': 0.005994842503189409, 'rf__n_estimators': 400, 'rf__max_depth': 20, 'mlp__hidden_layer_sizes': (50,), 'mlp__alpha': 0.01, 'lr__C': 1291.5496650148827, 'knn__n_neighbors': 3, 'et__n_estimators': 200, 'ada__n_estimators': 50, 'ada__learning_rate': 0.01}

In [28]:
X_train_scaled

NameError: name 'X_train_scaled' is not defined

In [30]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

# Define base models with tuned hyperparameters
base_models = [
    ('lr', LogisticRegression(C=1291.5496650148827, max_iter=5000, random_state=42)),  # Logistic Regression
    ('rf', RandomForestClassifier(n_estimators=400, max_depth=20, random_state=42)),   # Random Forest
    ('svc', SVC(kernel='rbf', C=0.005994842503189409, probability=True, random_state=42)),  # SVM
    ('knn', KNeighborsClassifier(n_neighbors=3)),                                      # K-Nearest Neighbors
    ('et', ExtraTreesClassifier(n_estimators=200, random_state=42)),                   # Extra Trees
    ('xgb', XGBClassifier(n_estimators=200, learning_rate=0.3, eval_metric='logloss', random_state=42)),  # XGBoost
    ('ada', AdaBoostClassifier(n_estimators=50, learning_rate=0.01, random_state=42)), # AdaBoost
    ('mlp', MLPClassifier(hidden_layer_sizes=(50,), alpha=0.01, max_iter=1000, random_state=42))  # Neural Network
]

# Define meta model
meta_model = LogisticRegression(max_iter=5000, random_state=42)

# Create the stacking classifier
stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5
)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Assuming X_train, X_test, y_train, y_test are already defined
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the stacking model
stacking_model.fit(X_train_scaled, Y_train)

# Evaluate the model
train_accuracy = stacking_model.score(X_train_scaled, Y_train)
test_accuracy = stacking_model.score(X_test_scaled, Y_test)

print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent thi

Train Accuracy: 0.9990
Test Accuracy: 0.6167


In [31]:
df1 = pd.read_csv('roberta_embeddings_english_abstractive.csv')

In [32]:
df1

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,Judgement Status
0,-0.045318,0.106291,-0.003215,-0.067849,0.027385,-0.127473,-0.029526,0.032465,0.032632,-0.086068,...,-0.050516,-0.105249,-0.060616,0.014517,0.111777,0.058287,-0.074195,0.009236,0.020355,1
1,-0.051724,0.116939,0.019695,-0.105494,0.033264,-0.070311,-0.006279,0.061977,0.007398,-0.091103,...,-0.015868,-0.088793,-0.080782,0.007715,0.105455,0.058583,-0.040270,0.008042,-0.013936,2
2,-0.010546,0.108884,0.001479,-0.138257,0.051355,-0.135540,0.008098,0.051294,0.010994,-0.042763,...,0.014642,-0.096850,-0.071336,-0.050518,0.096773,0.024355,-0.091703,-0.011808,0.005190,0
3,-0.054942,0.087816,0.019472,-0.118021,0.059199,-0.082324,-0.024870,0.060929,0.012410,-0.098716,...,-0.011953,-0.098715,-0.092882,-0.004579,0.122452,-0.007312,-0.053264,-0.000205,0.005530,0
4,-0.026756,0.077807,-0.000185,-0.094653,0.057685,-0.071129,-0.000685,0.097987,0.053599,-0.065829,...,-0.013227,-0.050602,-0.054803,-0.005757,0.072578,0.028736,-0.040133,-0.029626,0.037816,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,-0.052280,0.063353,-0.029092,-0.112168,0.026026,-0.086139,-0.025001,0.079336,0.038220,-0.089051,...,0.003848,-0.049991,-0.083095,0.011732,0.136884,0.000406,-0.103072,-0.027759,-0.016507,3
596,-0.040454,0.080203,-0.015394,-0.101792,0.036181,-0.041597,0.007757,0.047865,0.008642,-0.091039,...,0.021505,-0.032031,-0.064167,0.021279,0.122567,0.013715,-0.076298,-0.009738,-0.027242,2
597,-0.067318,0.111115,-0.009719,-0.082606,0.054640,-0.103232,-0.002115,0.021089,0.034756,-0.104129,...,0.019006,-0.062294,-0.049686,0.042695,0.105910,0.013915,-0.079819,-0.008745,0.007531,2
598,-0.064715,0.088847,-0.013383,-0.087596,0.054902,-0.089861,-0.038170,0.046950,-0.015348,-0.094655,...,0.003160,-0.061830,-0.096422,-0.021497,0.139260,0.059158,-0.117424,-0.034884,-0.042829,3


In [33]:
X1 = df1.iloc[:,:-1]

In [34]:
X1

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.045318,0.106291,-0.003215,-0.067849,0.027385,-0.127473,-0.029526,0.032465,0.032632,-0.086068,...,0.039978,-0.050516,-0.105249,-0.060616,0.014517,0.111777,0.058287,-0.074195,0.009236,0.020355
1,-0.051724,0.116939,0.019695,-0.105494,0.033264,-0.070311,-0.006279,0.061977,0.007398,-0.091103,...,0.012504,-0.015868,-0.088793,-0.080782,0.007715,0.105455,0.058583,-0.040270,0.008042,-0.013936
2,-0.010546,0.108884,0.001479,-0.138257,0.051355,-0.135540,0.008098,0.051294,0.010994,-0.042763,...,-0.014196,0.014642,-0.096850,-0.071336,-0.050518,0.096773,0.024355,-0.091703,-0.011808,0.005190
3,-0.054942,0.087816,0.019472,-0.118021,0.059199,-0.082324,-0.024870,0.060929,0.012410,-0.098716,...,0.020057,-0.011953,-0.098715,-0.092882,-0.004579,0.122452,-0.007312,-0.053264,-0.000205,0.005530
4,-0.026756,0.077807,-0.000185,-0.094653,0.057685,-0.071129,-0.000685,0.097987,0.053599,-0.065829,...,0.005963,-0.013227,-0.050602,-0.054803,-0.005757,0.072578,0.028736,-0.040133,-0.029626,0.037816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,-0.052280,0.063353,-0.029092,-0.112168,0.026026,-0.086139,-0.025001,0.079336,0.038220,-0.089051,...,-0.021647,0.003848,-0.049991,-0.083095,0.011732,0.136884,0.000406,-0.103072,-0.027759,-0.016507
596,-0.040454,0.080203,-0.015394,-0.101792,0.036181,-0.041597,0.007757,0.047865,0.008642,-0.091039,...,0.016783,0.021505,-0.032031,-0.064167,0.021279,0.122567,0.013715,-0.076298,-0.009738,-0.027242
597,-0.067318,0.111115,-0.009719,-0.082606,0.054640,-0.103232,-0.002115,0.021089,0.034756,-0.104129,...,-0.005077,0.019006,-0.062294,-0.049686,0.042695,0.105910,0.013915,-0.079819,-0.008745,0.007531
598,-0.064715,0.088847,-0.013383,-0.087596,0.054902,-0.089861,-0.038170,0.046950,-0.015348,-0.094655,...,-0.028029,0.003160,-0.061830,-0.096422,-0.021497,0.139260,0.059158,-0.117424,-0.034884,-0.042829


In [35]:
Y1 = df1.iloc[:,-1]

In [36]:
Y1

0      1
1      2
2      0
3      0
4      3
      ..
595    3
596    2
597    2
598    3
599    3
Name: Judgement Status, Length: 600, dtype: int64

In [37]:
X_train_df1,X_test_df1,Y_train_df1,Y_test_df1 = train_test_split(X1,Y1,test_size=0.2,random_state=42)

In [39]:
X_test_df1

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
110,-0.083560,0.070147,-0.015370,-0.103664,0.135056,-0.067583,-0.034615,-0.000115,0.058809,-0.072352,...,0.035152,-0.027006,-0.089304,-0.120331,-0.035402,0.093270,0.036072,-0.065274,-0.006414,-0.011368
419,-0.050680,0.073959,0.008506,-0.083283,0.054922,-0.107839,-0.010350,0.065205,0.052963,-0.070158,...,-0.005169,-0.000666,-0.032299,-0.049579,-0.024278,0.091060,0.037614,-0.090118,0.000911,0.022422
565,-0.040461,0.079744,-0.019714,-0.078833,0.078251,-0.064616,-0.014408,0.044760,0.053003,-0.088768,...,0.000069,-0.023669,-0.023824,-0.019322,0.026094,0.092766,0.014374,-0.067565,-0.014976,-0.002138
77,-0.052470,0.072993,0.010090,-0.092087,0.063448,-0.087107,-0.041693,0.034786,0.008224,-0.090344,...,0.017556,-0.019990,-0.054016,-0.091538,0.018257,0.126557,0.032525,-0.056932,-0.009042,-0.016341
181,-0.063083,0.057059,0.016751,-0.087716,0.077639,-0.072318,-0.020068,0.046868,0.024876,-0.085968,...,0.007339,-0.024612,-0.024292,-0.055543,-0.041920,0.097000,0.026627,-0.077174,0.012919,-0.024540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,-0.074427,0.064412,0.009134,-0.092475,0.093874,-0.013337,-0.014239,0.084721,0.003856,-0.103989,...,-0.012354,0.001333,-0.076812,-0.049030,-0.037104,0.083504,0.037495,-0.094041,0.007052,0.027278
340,-0.029502,0.060776,-0.015418,-0.071368,0.070629,-0.095759,-0.020478,0.022669,0.016587,-0.099918,...,0.019761,0.010205,-0.083346,-0.088746,-0.028311,0.113870,0.017377,-0.091353,0.017715,0.002801
148,-0.078855,0.068676,-0.009450,-0.105066,0.105090,-0.052190,0.000453,0.045122,-0.011150,-0.082455,...,-0.007636,0.001101,-0.082826,-0.071745,-0.056495,0.072523,0.010020,-0.126692,-0.003819,0.007998
494,-0.059255,0.072222,-0.001411,-0.103548,0.063705,-0.037711,-0.040557,0.057397,-0.000939,-0.123626,...,0.024937,-0.028124,-0.100507,-0.084598,-0.037331,0.131122,0.023067,-0.091684,0.032706,0.010656


In [41]:
stacking_model.predict(X_test_df1)

/opt/anaconda3/envs/may/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


ValueError: X has 768 features, but LogisticRegression is expecting 175 features as input.